## 1. Le modèle 

### 1.1 Principe

La régression logistique est un méthode d'apprentissage supervisée très utile pour expliquer / prédire une variable discrète (à l'opposé de continu) lorsque le jeu de données comporte de nombreuses variables catégorielles.

### 1.2 Notations

Soit Y la variable à prédire / expliquée (dans notre cas, il s'agit de la souscription ou non à l'option de deposit à long terme) et X = (X1, X2, ..., Xn) les variables explicatives

Dans le cadre de la regression logistique binaire, la variable Y prend deux modalités {0,1}. Les variables $X_{i}$ sont exclusivement binaires ou continues.

- Soit $\Omega$ un ensemble de n échantillons (ici n = 45211) comportant $n_{1}$ (respectivement $n_{0}$)) observations correspondant à la modalité 1 (respectivement 0) de Y (dans notre cas, $n_{1}$ = 5289 et $n_{0}$ = 39922).
- $P(Y=1)$ est la probabilité a priori pour que Y = 1. Pour simplifier on la notera p(1)
- $p(X \vert 1)$ est la distribution conditionnelle des X sachant la valeur prise par Y. 
- La probabilité a posteriori d'obtenir la modalité 1 de Y (resp. 0) sachant la valeur prise par X est $p(1 \vert X)$ (resp. $p(0 \vert X)$)


### 1.3 Le modèle LOGIT

On désigne par le terme LOGIT de $p(1 \vert X)$ l'expression suivante: $\ln {\frac  {p(1\vert X)}{1-p(1\vert X)}}=b_{0}+b_{1}x_{1}+...+b_{n}x_{n}$

- il s'agit bien d'une regression car on cherche à montrer une relation de dépendance entre une variable à expliquer tet une série de variables explicatives
- il s'agit d'une regression logistique car la loi de probabilité est modélisée par une loi logistique (https://fr.wikipedia.org/wiki/Fonction_logistique_(Verhulst))

#### Q.1 Pourquoi la fonction logistique est-elle intéressante ? Notamment par rapport à une fonction affine 

Réponse: 

Expliquer / prédire une variable binaire consiste alors à estimer au mieux les coefficients $b_{i}$ ci-dessus. Ensuite on peut retrouver la probabilité conditionnelle par la formule suivante: 

$p(1\vert X)={\frac  {e^{{b_{0}+b_{1}x_{1}+...+b_{n}x_{n}}}}{1+e^{{b_{0}+b_{1}x_{1}+...+b_{n}x_{n}}}}}$

Il existe plusieurs méthodes de resolutions / d'estimations de ces paramètres (voir principe du maximum de vraisemblance et algo méthode de résolution numérique Newton-Raphson). Nous allons nous concentrer plus sur l'interprétation des résultats dans le cas d'une classification plutot que sur les méthodes de résolution.

## 2. Implémentation

In [1]:
#!pip install preprocessing

### 2.1 Import des librairies et du jeu de données

In [2]:
import pandas as pd
import numpy as np
import preprocessing
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn import linear_model

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

On réutilise le jeu de données sur le marketing dans la banque

In [3]:
df = pd.read_csv("bank-full.csv",  sep = ";")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


### 2.2 Preprocessing des données (voir le package en détails)

Comme précisé plus haut, on a besoin de variables exclusivement binaires ou continues. 

#### Q.2 Quel type d'encoding des variables catégorielles doit-on mettre en place pour ce faire ? 

Réponse: quantitative

#### Consigne 1: Mettre en place cet encoding 

Vous pouvez faire cet encoding directement via des instructions dans le notebook. Mais le mieux étant de créer un package dans un script python en dehors du notebook pour faire ce processing et appeler ensuite ce package dans le notebook (comme si vous appeliez un package de pandas ou numpy)

In [10]:
df_binaire = df.copy()
df_binaire.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [12]:
categorical_data = ['default', 'housing', 'loan', 'y']

for i in categorical_data:
    df_binaire[i] = df_binaire[i].replace('yes',1).replace('no',0)

In [13]:
df_binaire = pd.get_dummies (data = df_binaire, columns = ['job', 'marital', 'education', 'poutcome', 'month', 'contact'],
                             prefix = ['job', 'marital', 'education', 'poutcome', 'month', 'contact'], drop_first = True)
#le drop_first  elimine une colone sur chacune des variable aprés le get dummies

In [14]:
df_binaire

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,y,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,poutcome_other,poutcome_success,poutcome_unknown,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,contact_telephone,contact_unknown
0,58,0,2143,1,0,5,261,1,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
1,44,0,29,1,0,5,151,1,-1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
2,33,0,2,1,1,5,76,1,-1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
3,47,0,1506,1,0,5,92,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
4,33,0,1,0,0,5,198,1,-1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,0,825,0,0,17,977,3,-1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
45207,71,0,1729,0,0,17,456,2,-1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
45208,72,0,5715,0,0,17,1127,5,184,3,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
45209,57,0,668,0,0,17,508,4,-1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0


#### Consigne 2: Séparer l'échantillon de données en base d'apprentissage de base de test (70% / 30%)

In [15]:
df_binaire_explicative = df_binaire.drop(['y'], axis = 1)
df_binaire_explicative #version de df_binaire sans la colone y dont on se servira a part.

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,poutcome_other,poutcome_success,poutcome_unknown,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,contact_telephone,contact_unknown
0,58,0,2143,1,0,5,261,1,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
1,44,0,29,1,0,5,151,1,-1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
2,33,0,2,1,1,5,76,1,-1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
3,47,0,1506,1,0,5,92,1,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
4,33,0,1,0,0,5,198,1,-1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45206,51,0,825,0,0,17,977,3,-1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
45207,71,0,1729,0,0,17,456,2,-1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
45208,72,0,5715,0,0,17,1127,5,184,3,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
45209,57,0,668,0,0,17,508,4,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0


In [16]:
X = df_binaire_explicative
y = df_binaire.y #c'est ici que le y sert
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [19]:
X_test

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,marital_married,marital_single,education_secondary,education_tertiary,education_unknown,poutcome_other,poutcome_success,poutcome_unknown,month_aug,month_dec,month_feb,month_jan,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,contact_telephone,contact_unknown
3776,40,0,580,1,0,16,192,1,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
9928,47,0,3644,0,0,9,83,2,-1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1
33409,25,0,538,1,0,20,226,1,-1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
31885,42,0,1773,0,0,9,311,1,336,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
15738,56,0,217,0,1,21,121,2,-1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25573,39,0,3494,1,0,19,139,1,92,4,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6395,58,0,-37,1,0,27,140,2,-1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1
24513,38,0,0,1,0,17,213,2,-1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
34048,31,0,2915,0,0,30,228,1,86,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


#### Consigne 3: Utiliser la fonction de scikit learn permettant de mettre en place une régression logistique pour entrainer un modèle 

In [21]:
from sklearn.linear_model import LogisticRegression # import pour la regression logistique

In [22]:
modele_logit = LogisticRegression(penalty='none',solver='newton-cg') # pour linstant on laisse le warning, on s'en occupera plus tard
modele_logit.fit(X_train,y_train)

C:\Users\ryven\Anaconda3\lib\site-packages\sklearn\utils\optimize.py:203: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='none',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)

#### Consigne 4: Faire des prédictions sur l'échantillon de test

In [25]:
modele_logit.predict(X_train)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

#### Consigne 5: Créez une matrice de confusion pour analyser les résultats

In [26]:
y_pred = modele_logit.predict(X_train)

y_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [28]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train, y_pred)

array([[26095,   652],
       [ 2295,  1249]], dtype=int64)

#### Q.3: Combien a-t'on de TP (True Positif), TN (True Negatif), FP (False Positive) et FN (False Negative) ? 

In [32]:
tn, fp, fn, tp = confusion_matrix(y_train, y_pred).ravel()
print("tn =", tn,"fp =", fp,"fn =", fn,"tp =", tp)

tn = 26095 fp = 652 fn = 2295 tp = 1249


#### Q.4 Quelle est l'accuracy du modèle ? Sa précision ? Son recall ? Son F-1 score ?

In [36]:
print("accuracy =",(tn+tp)/(tn+fn+tp+fp))

accuracy = 0.9027103760192796


In [37]:
print("precision =",tp/(tp+fp))

precision = 0.6570226196738559


In [38]:
print("recall =",(tp)/(fn+tp))

recall = 0.35242663656884876


In [42]:
from sklearn.metrics import f1_score #on peut en fait faire ça pour les trois calcules du dessus, accuracy_score, precision_score, recall_score.
print("F1 =",f1_score(y_train, y_pred)) #et c'est probablement que le calcule du dessus

F1 = 0.45876951331496785


#### Consigne 6: Construire une courbe ROC et une courbe Precision - Recall à partir des résultats

In [ ]:
plt.figure() # la j'y travaille encore c'est pas parametré 
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

#### Consigne 7: Faire des prédictions sur l'échantillon d'apprentissage et calculer toutes les métriques ci-dessus

#### Consigne 8: Comparer avec les résultats des prédictions sur l'échantillon d'apprentissage et l'échantillon de test 

#### Q.5 Observe t'on un phénomène particulier ? 

#### Bonus: modifier les hyperparamètres du modèle pour essayer d'obtenir de meilleurs résultats

# Précision c'est la vérité positive sur la prédiction positive


# Formule : True Positive / (False Positive + True Positive)